# Data Mining Midterm Project

# Importing Libraries

In [1]:
import numpy as np 
import pandas as pd
import itertools
from itertools import combinations

# Importing Datasets

In [2]:
data=pd.read_csv(r'C:\Akhil\NJIT Courses\Data Mining\Grocery Store.csv')
print('You selected Grocery Store dataset')

You selected Grocery Store dataset


# Data 

In [3]:
print(data)

   milk bread biscuit cornflakes bournvita  jam maggi  tea coffee cock sugar
0     t     t       t        NaN       NaN  NaN   NaN  NaN    NaN  NaN   NaN
1     t     t       t          t       NaN  NaN   NaN  NaN    NaN  NaN   NaN
2   NaN     t     NaN        NaN         t  NaN   NaN    t    NaN  NaN   NaN
3     t     t     NaN        NaN       NaN    t     t  NaN    NaN  NaN   NaN
4   NaN   NaN       t        NaN       NaN  NaN     t    t    NaN  NaN   NaN
5   NaN     t     NaN        NaN         t  NaN   NaN    t    NaN  NaN   NaN
6   NaN   NaN     NaN          t       NaN  NaN     t    t    NaN  NaN   NaN
7   NaN     t       t        NaN       NaN  NaN     t    t    NaN  NaN   NaN
8   NaN     t     NaN        NaN       NaN    t     t    t    NaN  NaN   NaN
9     t     t     NaN        NaN       NaN  NaN   NaN  NaN    NaN  NaN   NaN
10  NaN   NaN       t          t       NaN  NaN   NaN  NaN      t    t   NaN
11  NaN   NaN       t          t       NaN  NaN   NaN  NaN      t    t   NaN

In [4]:
df = data

In [6]:
print(df.head())

  milk bread biscuit cornflakes bournvita  jam maggi  tea coffee cock sugar
0    t     t       t        NaN       NaN  NaN   NaN  NaN    NaN  NaN   NaN
1    t     t       t          t       NaN  NaN   NaN  NaN    NaN  NaN   NaN
2  NaN     t     NaN        NaN         t  NaN   NaN    t    NaN  NaN   NaN
3    t     t     NaN        NaN       NaN    t     t  NaN    NaN  NaN   NaN
4  NaN   NaN       t        NaN       NaN  NaN     t    t    NaN  NaN   NaN


# Indexing Each Item from the header of the data file

In [7]:
item_list = list(df.columns)
item_dict = dict()

for i, item in enumerate(item_list):
    item_dict[item] = i + 1

item_dict

{'milk': 1,
 'bread': 2,
 'biscuit': 3,
 'cornflakes': 4,
 'bournvita': 5,
 'jam': 6,
 'maggi': 7,
 'tea': 8,
 'coffee': 9,
 'cock': 10,
 'sugar': 11}

# Extracting Transactions from the Data

In [8]:
transactions = list()

for i, row in df.iterrows():
    transaction = set()
    
    for item in item_dict:
        if row[item] == 't':
            transaction.add(item_dict[item])
    transactions.append(transaction)

In [9]:
transactions

[{1, 2, 3},
 {1, 2, 3, 4},
 {2, 5, 8},
 {1, 2, 6, 7},
 {3, 7, 8},
 {2, 5, 8},
 {4, 7, 8},
 {2, 3, 7, 8},
 {2, 6, 7, 8},
 {1, 2},
 {3, 4, 9, 10},
 {3, 4, 9, 10},
 {5, 9, 11},
 {2, 9, 10},
 {2, 3, 11},
 {4, 9, 11},
 {2, 5, 11},
 {2, 9, 11},
 {2, 9, 11},
 {1, 4, 8, 9}]

# Get Support Function that evaluates the support value for a set given all the transactions

In [171]:
def get_support(transactions, item_set):
    match_count = 0
    for transaction in transactions:
        if item_set.issubset(transaction):
            match_count += 1
            
    return float(match_count/len(transactions))

# self_join performs join based on the last level valid sets. It joins each sets together by performing union and if the length exceeds the current level, it will skip that set.

In [172]:
def self_join(frequent_item_sets_per_level, level):
    current_level_candidates = list()
    last_level_items = frequent_item_sets_per_level[level - 1]
    
    if len(last_level_items) == 0:
        return current_level_candidates
    
    for i in range(len(last_level_items)):
        for j in range(i+1, len(last_level_items)):
            itemset_i = last_level_items[i][0]
            itemset_j = last_level_items[j][0]
            union_set = itemset_i.union(itemset_j)
            
            if union_set not in current_level_candidates and len(union_set) == level:
                current_level_candidates.append(union_set)
                
    return current_level_candidates

In [173]:
def get_single_drop_subsets(item_set):
    single_drop_subsets = list()
    for item in item_set:
        temp = item_set.copy()
        temp.remove(item)
        single_drop_subsets.append(temp)
        
    return single_drop_subsets

def is_valid_set(item_set, prev_level_sets):
    single_drop_subsets = get_single_drop_subsets(item_set)
    
    for single_drop_set in single_drop_subsets:
        if single_drop_set not in prev_level_sets:
            return False
    return True

def pruning(frequent_item_sets_per_level, level, candidate_set):
    post_pruning_set = list()
    if len(candidate_set) == 0:
        return post_pruning_set
    
    prev_level_sets = list()
    for item_set, _ in frequent_item_sets_per_level[level - 1]:
        prev_level_sets.append(item_set)
        
    for item_set in candidate_set:
        if is_valid_set(item_set, prev_level_sets):
            post_pruning_set.append(item_set)
            
    return post_pruning_set

# This is the main function which uses all the above described Utility functions to implement the Apriori Algorithm and generate the list of frequent itemsets for each level for the provided transactions and min_support value.

In [174]:
from collections import defaultdict
def apriori(min_support):
    frequent_item_sets_per_level = defaultdict(list)
    print("level : 1", end = " ")
    
    for item in range(1, len(item_list) + 1):
        support = get_support(transactions, {item})
        if support >= min_support:
            frequent_item_sets_per_level[1].append(({item}, support))
        
    for level in range(2, len(item_list) + 1):
        print(level, end = " ")
        current_level_candidates = self_join(frequent_item_sets_per_level, level)

        post_pruning_candidates = pruning(frequent_item_sets_per_level, level, current_level_candidates)
        if len(post_pruning_candidates) == 0:
            break

        for item_set in post_pruning_candidates:
            support = get_support(transactions, item_set)
            if support >= min_support:
                frequent_item_sets_per_level[level].append((item_set, support))
                
    return frequent_item_sets_per_level

# Entering the Minimum Support Value

In [175]:
print("Enter the Minimum Support : ")
min_support = input()
min_support = (min_support)/100
frequent_item_sets_per_level = apriori(min_support)

Enter the Minimum Support :  20
level : 1 2 3 

# The below code produces a dictionary called item_support_dict which from frequent_item_sets_per_level that maps items to their support values

In [178]:
item_support_dict = dict()
item_list = list()

key_list = list(item_dict.keys())
val_list = list(item_dict.values())

for level in frequent_item_sets_per_level:
    for set_support_pair in frequent_item_sets_per_level[level]:
        for i in set_support_pair[0]:
            item_list.append(key_list[val_list.index(i)])
        item_support_dict[frozenset(item_list)] = set_support_pair[1]
        item_list = list()

# The find_subset function takes the item and item_length as parameter and it returns all the possible combinations of elements inside the items

In [179]:
def find_subset(item, item_length):
    combs = []
    for i in range(1, item_length + 1):
        combs.append(list(combinations(item, i)))
        
    subsets = []
    for comb in combs:
        for elt in comb:
            subsets.append(elt)
            
    return subsets

# This function generates the association rules in accordance withe the minimum confidence value and the provided dictionary of itemsets against their support values. It takes the mininmum confidence value and support_dict as a parameter, and returns rules as a list.

In [180]:
def association_rules(min_confidence, support_dict):
    rules = list()
    for item, support in support_dict.items():
        item_length = len(item)
       
        if item_length > 1:
            subsets = find_subset(item, item_length)
           
            for A in subsets:
                B = item.difference(A)
               
                if B:
                    A = frozenset(A)
                    
                    AB = A | B
                    
                    confidence = support_dict[AB] / support_dict[A]
                    if confidence >= min_confidence:
                        rules.append((A, B, confidence))
    
    return rules

In [181]:
print("Enter the Minimum Confidence : ")
confidence = input()
min_confidence = (confidence)/100
association_rules = association_rules(min_confidence, support_dict = item_support_dict)

Enter the Minimum Confidence :  30


# Output

In [182]:
print("Number of rules: ", len(association_rules))

for rule in association_rules:
    print('{0} -> {1} <confidence: {2}>'.format(set(rule[0]), set(rule[1]), rule[2]))


Number of rules:  14 

{'milk'} -> {'bread'} <confidence: 0.8>
{'bread'} -> {'milk'} <confidence: 0.3076923076923077>
{'bread'} -> {'biscuit'} <confidence: 0.3076923076923077>
{'biscuit'} -> {'bread'} <confidence: 0.5714285714285715>
{'bread'} -> {'tea'} <confidence: 0.3076923076923077>
{'tea'} -> {'bread'} <confidence: 0.5714285714285715>
{'bread'} -> {'sugar'} <confidence: 0.3076923076923077>
{'sugar'} -> {'bread'} <confidence: 0.6666666666666667>
{'cornflakes'} -> {'coffee'} <confidence: 0.6666666666666667>
{'coffee'} -> {'cornflakes'} <confidence: 0.5>
{'maggi'} -> {'tea'} <confidence: 0.8>
{'tea'} -> {'maggi'} <confidence: 0.5714285714285715>
{'coffee'} -> {'sugar'} <confidence: 0.5>
{'sugar'} -> {'coffee'} <confidence: 0.6666666666666667>
